# **Replicating LeNet and AlexNet in Tensorflow2.0 using Keras**

---

Using **Keras with a TensorFlow 2.0** Backend to replicate both **LeNet and AlexNet** in Keras and train it to **recognize handwritten digits in the MNIST dataset and the 10 images classes of CIFAR10**
1. Replicate the LeNet CNN Architecture 
2. Replicate the AlexNet CNN Architecture 

## **Let's construct LeNet in Keras**

![](https://www.researchgate.net/profile/Sheraz_Khan8/publication/321586653/figure/fig4/AS:568546847014912@1512563539828/The-LeNet-5-Architecture-a-convolutional-neural-network.png)
## **LeNet Architecture**
S.No | Layers | Output Shape (Height, Width, Channels)
--- | --- | ---
1 | Input Layer | 32 x 32 x 1
2 | Conv2d [6 Filters of size = 5x5, stride = 1, padding = 0 ] | 28 x 28 x 6
3 | Average Pooling [stride = 2, padding = 0] | 14 x 14 x 6
4 | Conv2d [16 Filters of size = 5x5, stride = 1, padding = 0 ] | 10 x 10 x 16
5 | Average Pooling [stride = 2, padding = 0] | 5 x 5 x 16
6 | Conv2d [120 Filters of size = 5x5, stride = 1, padding = 0 ] | 1 x 1 x 120
7 | Linear1 Layer | 120 
8 | Linear2 Layer | 84 
9 | Final Linear Layer | 10



### **1. Loading and preprocessing our Data**

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

# Loads the MNIST dataset
(x_train, y_train), (x_test, y_test)  = mnist.load_data()

# Store the number of rows and columns
img_rows = x_train[0].shape[0]
img_cols = x_train[1].shape[0]

# Reshape our dataset
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

# Store the shape of a single image 
input_shape = (img_rows, img_cols, 1)

# Change our image type to float32 data type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize our data by changing the range from (0 to 255) to (0 to 1)
x_train /= 255
x_test /= 255

# One hot encode outputs
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_classes = y_test.shape[1]
num_pixels = x_train.shape[1] * x_train.shape[2]

### **2. Building layers to replicate LeNet**

In [ ]:
kernel_size = (5, 5)
pool_size = (2, 2)

# Create model
model = Sequential()

# Sets CRP (Convulation, RELU, Pooling)

# First layer
model.add(Conv2D(6, kernel_size, padding = "same", input_shape = input_shape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size, strides = (2, 2)))

# Second layer
model.add(Conv2D(16, kernel_size, padding = "same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size, strides = (2, 2)))

# Third layer
model.add(Conv2D(120, kernel_size, padding = "same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size, strides = (2, 2)))

# Fully connected layers (w/ RELU)
model.add(Flatten())
model.add(Dense(120))
model.add(Activation("relu"))
model.add(Dense(84))
model.add(Activation("relu"))

# Softmax (for classification)
model.add(Dense(num_classes))
model.add(Activation("softmax"))

# Define optimizer
model.compile(loss = 'categorical_crossentropy', optimizer = Adadelta(), metrics = ['accuracy'])
    
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 28, 28, 6)         156       
                                                                 
 activation_14 (Activation)  (None, 28, 28, 6)         0         
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 14, 14, 6)        0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 14, 14, 16)        2416      
                                                                 
 activation_15 (Activation)  (None, 14, 14, 16)        0         
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 7, 7, 16)         0         
 2D)                                                  

### **3. Train LeNet on our MNIST Dataset**

In [ ]:
# Training Parameters
epochs = 50
batch_size = 128

# Our train function
history = model.fit(x_train, 
                    y_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    validation_data = (x_test, y_test),
                    shuffle = True)

# Save our model
model.save("mnist_LeNet.h5")

# Evaluate the performance of our trained model
scores = model.evaluate(x_test, y_test, verbose=1)

print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/50
469/469 [==============================] - 3s 6ms/step - loss: 2.2964 - accuracy: 0.1413 - val_loss: 2.2879 - val_accuracy: 0.1642
Epoch 2/50
469/469 [==============================] - 3s 5ms/step - loss: 2.2792 - accuracy: 0.1927 - val_loss: 2.2699 - val_accuracy: 0.2362
Epoch 3/50
128/469 [=======>......................] - ETA: 1s - loss: 2.2672 - accuracy: 0.2512

KeyboardInterrupt: ignored

## **Replicate AlexNET and train it on the CIFAR10 Dataset**

AlexNet was the 2012 ImageNet winner achieved a top-5 error of 15.3%, more than 10.8 percentage points lower than that of the runner up!

![](https://paperswithcode.com/media/methods/Screen_Shot_2020-06-22_at_6.35.45_PM.png)

![](https://production-media.paperswithcode.com/datasets/CIFAR-10-0000000431-b71f61c0_U5n3Glr.jpg)

In [ ]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.utils  import to_categorical
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

# Loads the CIFAR dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Display our data shape/dimensions
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Now we one hot encode outputs
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


### **Create our layers to replicate AlexNet**

In [ ]:
l2_reg = 0.001

# Create model
model = Sequential()

# 1st Conv Layer 
model.add(Conv2D(96, (11, 11), input_shape=x_train.shape[1:], padding='same', kernel_regularizer=l2(l2_reg)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 2nd Conv Layer 
model.add(Conv2D(256, (5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 3rd Conv Layer 
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 4th Conv Layer 
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(1024, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 5th Conv Layer 
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(1024, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 1st FC Layer
model.add(Flatten())
model.add(Dense(3072))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 2nd FC Layer
model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 3rd FC Layer
model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

print(model.summary())

model.compile(loss = 'categorical_crossentropy', optimizer = Adadelta(), metrics = ['accuracy'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 32, 32, 96)        34944     
                                                                 
 batch_normalization_9 (Batc  (None, 32, 32, 96)       384       
 hNormalization)                                                 
                                                                 
 activation_21 (Activation)  (None, 32, 32, 96)        0         
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 16, 16, 96)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 16, 16, 256)       614656    
                                                                 
 batch_normalization_10 (Bat  (None, 16, 16, 256)     

### **Train AlexNet on our CIFAR10 Dataset**

In [ ]:
# Training Parameters
epochs = 25
batch_size = 128

history = model.fit(x_train, 
                    y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    shuffle=True)

# Save our model
model.save("CIFAR10_AlexNet_10_Epoch.h5")


# Evaluate the performance of our trained model
scores = model.evaluate(x_test, y_test, verbose=1)

print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/25
1563/1563 [==============================] - 115s 73ms/step - loss: 1.8567 - accuracy: 0.3988 - val_loss: 1.6850 - val_accuracy: 0.4822
Epoch 2/25
1563/1563 [==============================] - 118s 76ms/step - loss: 1.7356 - accuracy: 0.4576 - val_loss: 1.6122 - val_accuracy: 0.5095
Epoch 3/25
1563/1563 [==============================] - 118s 76ms/step - loss: 1.6573 - accuracy: 0.4913 - val_loss: 1.5654 - val_accuracy: 0.5314
Epoch 4/25
1563/1563 [==============================] - 118s 76ms/step - loss: 1.6048 - accuracy: 0.5185 - val_loss: 1.5313 - val_accuracy: 0.5462
Epoch 5/25
1563/1563 [==============================] - 119s 76ms/step - loss: 1.5579 - accuracy: 0.5420 - val_loss: 1.5040 - val_accuracy: 0.5648
Epoch 6/25
1563/1563 [==============================] - 118s 76ms/step - loss: 1.5235 - accuracy: 0.5596 - val_loss: 1.4791 - val_accuracy: 0.5748
Epoch 7/25
1563/1563 [==============================] - 118s 76ms/step - loss: 1.4828 - accuracy: 0.5825 - val_loss: 1